### Real Time Data Analytics | Lab
Submitted by <b> Paraneetharan M</b>

This Lab submission consists of 2 parts.
* Part - 1: Spark Streaming to listen a local host port and Calculates Quadrant counts
* Part - 2: Socket to generate Quadrant point automatically

## Part 1: Spark Streaming

In [105]:
# Spark Streaming Libs
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

import sys
from ast import literal_eval

In [146]:
# Create a Spark Context
sc = SparkContext(appName="parani_SparkStreaming_Lab")

In [147]:
# Create a Streaming Context
sc.setLogLevel("WARN")
ssc = StreamingContext(sc,10)

In [148]:
# Setup Checkpoint
ssc.checkpoint("checkpoint")

In [149]:
# Func to get Quadrant of (x,y)
# SparkStreaming usually returns values in bytestring. 
# So it is necessary to convert point from string to tuple
def getQuadrant(point):
    try:
        # Convert ByteString to tuple
        (x,y) = literal_eval(point)
    except:
        return("Unknown", 1)
    
    # (+,+)
    if x > 0 and y > 0: 
        return ("First Quadrant", 1)  
    # (+,-)
    elif x > 0 and y < 0:
        return ("Fourth Quadrant", 1)
    
    # (-,-)
    elif x < 0 and y < 0:
        return ("Third Quadrant", 1)
    
    # (-,+)
    elif x < 0 and y > 0:
        return ("Second Quadrant", 1)
    
    #(0,+/-)
    elif x == 0 and y != 0:
        return ("on X Axis", 1)
    
    #(+/-,0)
    elif x != 0 and y == 0:
        return ("on Y Axis", 1)
    
    #(0,0)
    else:
        return ("Origin", 1)

In [150]:
# Func to add all values
def updateFunction(new_values, last_sum):
    return sum(new_values) + (last_sum or 0)

In [151]:
# Listen to localhost port 6868
lines = ssc.socketTextStream("localhost", 6767)
lines.pprint()

In [152]:
# Calculate the all time count
points_count = lines.map(getQuadrant).updateStateByKey(updateFunction)
points_count.pprint()

In [153]:
if __name__ == "__main__":
    try:
        ssc.start()
        ssc.awaitTerminationOrTimeout(1000)
        ssc.stop(stopGraceFully=True,stopSparkContext=True)
    
    except KeyboardInterrupt:
        ssc.stop(stopGraceFully=True,stopSparkContext=True)

-------------------------------------------
Time: 2018-09-02 07:33:50
-------------------------------------------
(-1, -5)
(-4, 8)

-------------------------------------------
Time: 2018-09-02 07:33:50
-------------------------------------------
('Second Quadrant', 1)
('Third Quadrant', 1)

-------------------------------------------
Time: 2018-09-02 07:34:00
-------------------------------------------
(6, 8)
(-5, -5)
(-7, 3)
(-6, 2)
(4, -3)

-------------------------------------------
Time: 2018-09-02 07:34:00
-------------------------------------------
('Second Quadrant', 3)
('Third Quadrant', 2)
('First Quadrant', 1)
('Fourth Quadrant', 1)

-------------------------------------------
Time: 2018-09-02 07:34:10
-------------------------------------------
(8, -3)
(0, 1)
(-4, -9)
(-6, -5)
(7, 0)

-------------------------------------------
Time: 2018-09-02 07:34:10
-------------------------------------------
('Second Quadrant', 3)
('Third Quadrant', 4)
('First Quadrant', 1)
('on X Axis'